In [ ]:
%pip install mediapipe dlib



# Code to generate OFI images for model input

## Visualizing output of a sample video

In [ ]:
import cv2
import dlib
import os

# Load video
video_path = ''
cap = cv2.VideoCapture(video_path)

# Initialize dlib face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor_path = "C:\\Users\\VB\\Downloads\\shape_predictor_68_face_landmarks.dat"  # <-- Update path if needed
predictor = dlib.shape_predictor(predictor_path)

# Output directory to save FLI images
output_dir = "FLI_output"
os.makedirs(output_dir, exist_ok=True)

frame_count = 0
max_frames = 128

while cap.isOpened():
    ret, frame = cap.read()
    if not ret or frame_count >= max_frames:
        break

    # Process every 4th frame
    if frame_count % 4 != 0:
        frame_count += 1
        continue

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)
        for i in range(68):
            x = landmarks.part(i).x
            y = landmarks.part(i).y
            cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

    # Save or display frame
    output_path = os.path.join(output_dir, f"fli_frame_{frame_count}.jpg")
    cv2.imwrite(output_path, frame)

    # Optional: Display
    cv2.imshow("Facial Landmarks", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_count += 1

cap.release()
cv2.destroyAllWindows()


## Store the video path for no-pain (BL1) and pain (PA4) files

In [ ]:
import os

# Function to walk through the directory and find mp4 files containing "BL1"
def find_bl1_mp4_files(directory):
    bl1_files = []

    # Walk through all files and subdirectories
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file is an mp4 and contains "BL1" in the name
            if file.endswith(".mp4") and "PA4" in file:
                # Get the full path of the file
                full_file_path = os.path.join(root, file)
                bl1_files.append(full_file_path)

    return bl1_files

# Example usage: Provide your directory path
directory_path = ''  #Replace with your directory path

# Get list of mp4 files containing "PA4"
bl1_mp4_files = find_bl1_mp4_files(directory_path)

# Display the list
for file in bl1_mp4_files:
    print(file)

## Code to process video and same image files

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp

# List of video file paths
video_paths = bl1_mp4_files  # Ensure this variable is defined

# Output directory
output_dir = ''
os.makedirs(output_dir, exist_ok=True)

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5)

# Landmark indices to keep (excluding chin/jawline)
valid_indices = list(range(17, 68))  # Eyebrows, eyes, nose, and mouth

# Exaggeration factor
exaggeration_factor = 4

# Process each video
for video_path in video_paths:
    video_name = os.path.basename(video_path).split('.')[0]
    print(f"Processing video: {video_name}")

    video_output_dir = os.path.join(output_dir, video_name)
    os.makedirs(video_output_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)

    frame_count = 0
    original_landmarks = None
    previous_landmarks = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % 4 != 0 or frame_count > 132:
            frame_count += 1
            continue

        frame_count2 = frame_count - 4
        image_filename = f"{video_name}_Landmark_{frame_count2}.png"
        image_filepath = os.path.join(video_output_dir, image_filename)

        # Skip processing if image already exists
        if os.path.exists(image_filepath):
            frame_count += 1
            continue

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_frame)

        if results.multi_face_landmarks:
            for landmarks in results.multi_face_landmarks:
                # Extract and filter landmark coordinates
                landmark_coords = np.array([
                    (int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0]))
                    for landmark in landmarks.landmark
                ])
                valid_landmarks = landmark_coords[valid_indices]

                # Store first frame landmarks for long-term movement
                if original_landmarks is None:
                    original_landmarks = valid_landmarks.copy()

                # Calculate long-term movement
                long_term_move = (valid_landmarks - original_landmarks) * exaggeration_factor

                # Calculate short-term movement if not first frame
                if previous_landmarks is not None:
                    short_term_move = (valid_landmarks - previous_landmarks) * exaggeration_factor

                    # Save quiver plot
                    plt.figure(figsize=(2.56, 2.56), dpi=100)  # 256x256 resolution

                    # Plot long-term vectors (from original landmarks) in red
                    plt.quiver(original_landmarks[:, 0], original_landmarks[:, 1],
                               long_term_move[:, 0], long_term_move[:, 1],
                               angles='xy', scale_units='xy', scale=1, color='red',
                               width=0.005, headwidth=3, headlength=5, headaxislength=4)

                    # Plot short-term vectors (from previous landmarks) in blue
                    plt.quiver(previous_landmarks[:, 0], previous_landmarks[:, 1],
                               short_term_move[:, 0], short_term_move[:, 1],
                               angles='xy', scale_units='xy', scale=1, color='blue',
                               width=0.005, headwidth=3, headlength=5, headaxislength=4)

                    plt.gca().invert_yaxis()
                    plt.axis('off')
                    plt.grid(False)
                    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
                    plt.savefig(image_filepath, dpi=100, bbox_inches='tight', pad_inches=0)
                    plt.close()

                previous_landmarks = valid_landmarks

        frame_count += 1

    cap.release()

print("MediaPipe landmark movement images with long-term (red) and short-term (blue) vectors saved.")
